In [1]:
%matplotlib notebook
import time
import numpy as np
import scipy as sp
import ipywidgets as widgets
from inspect import signature
from ipywidgets import interact, interact_manual
from IPython.display import display
from lolimot import LolimotRegressor
from plot import PlotterLLM

# Local Model Playground

In [2]:
# Benutzerdefnierte Funktionen
functions=dict(
    arctan=lambda u: 3 * np.arctan(2*u) + 5,
    poly=lambda u: (4+u)*(u+1)*(u-1)*(u-3)/14,
    trigonometric=lambda u: np.sin(2*np.pi*u/4) + 0.5 * np.cos(2*np.pi*u),
    affensattel=lambda u1, u2: (u1**3 - 3 * u1 * u2**2 ) / 200)

In [3]:
items = [widgets.ToggleButton(value=True, description='result', disabled=False, tooltip='Description'),
         widgets.ToggleButton(value=True, description='error', disabled=False, tooltip='Description'),
         widgets.ToggleButton(value=False, description='validity_function', disabled=False, tooltip='Description'),
         widgets.ToggleButton(value=False, description='models', disabled=False, tooltip='Description'),
         widgets.ToggleButton(value=False, description='report', disabled=False, tooltip='Description')]

def lolimot(model_complexity, p, x_range, func, k):
    # TRAINING
    llm = LolimotRegressor(model_complexity=model_complexity, p=p)
    
    if len(signature(func).parameters) == 2:
        x1, x2 = np.meshgrid(np.linspace(-5, 5, k), np.linspace(-5, 5, k))
        y = np.reshape(func(x1, x2), (k*k, ))
        X = np.vstack((x1.flatten(), x2.flatten())).T
        
        _ = llm.fit(X, y)
    
        # PLOT
        k = int(np.sqrt(k) * 8)
        x1, x2 = np.meshgrid(np.linspace(-5, 5, k), np.linspace(-5, 5, k))
        y = np.reshape(func(x1, x2), (k*k, ))
        x = [x1, x2]
        
    else:
        x = np.reshape(np.linspace(x_range[0], x_range[1], k), (k, 1))
        y = func(x).ravel()
        _ = llm.fit(x, y)
    
        # PLOT
        k = k*2
        x = np.reshape(np.linspace(x_range[0], x_range[1], k), (k, 1))
        y = func(x)

    plotter = PlotterLLM(options=[item.description for item in items if item.value])
    plotter.update(x, y, llm)

# Widgets
complexity_range = widgets.IntSlider(
    continuous_update=False, min=1, max=1000,
    layout=widgets.Layout(width='90%'),
    description="complexity")
p_range = widgets.FloatSlider(
    continuous_update=False, min=0.05, max=1.5,
    value=0.33, step=0.01, layout=widgets.Layout(width='90%'))
x_range = widgets.IntRangeSlider(
    continuous_update=False, min=-20, max=20,
    value=[-5, 5], layout=widgets.Layout(width='90%'),
    description="range")
functions = widgets.Dropdown(
    options=functions)
sample_num = widgets.IntText(value=100, description='number of samples:', disabled=False)


print("Push 'Run Interact' to start:")
plot_options = widgets.Box([widgets.Label(value="plot options:")] + items)
display(plot_options)
interact_manual(lolimot, model_complexity=complexity_range, p=p_range, x_range=x_range, func=functions, k=sample_num);

Push 'Run Interact' to start


Box(children=(Label(value='plot options:'), ToggleButton(value=True, description='result', tooltip='Description'), ToggleButton(value=True, description='error', tooltip='Description'), ToggleButton(value=False, description='validity_function', tooltip='Description'), ToggleButton(value=False, description='models', tooltip='Description'), ToggleButton(value=False, description='report', tooltip='Description')))

interactive(children=(IntSlider(value=1, continuous_update=False, description='complexity', layout=Layout(width='90%'), max=1000, min=1), FloatSlider(value=0.33, continuous_update=False, description='p', layout=Layout(width='90%'), max=1.5, min=0.05, step=0.01), IntRangeSlider(value=(-5, 5), continuous_update=False, description='range', layout=Layout(width='90%'), max=20, min=-20), Dropdown(description='func', options={'arctan': <function <lambda> at 0x7f79c63159d8>, 'poly': <function <lambda> at 0x7f79c6315a60>, 'trigonometric': <function <lambda> at 0x7f79c6315ae8>, 'affensattel': <function <lambda> at 0x7f79c6315b70>}, value=<function <lambda> at 0x7f79c63159d8>), IntText(value=100, description='number of samples:'), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))